# Lesson 2: Practical Deep Learning for Coders - Deployment

This notebook presents my personal notes and work coming from my experience with the second lesson of the course "**Practical Deep Learning**" by Jeremy Howard ([here](https://www.youtube.com/watch?v=F4tvM4Vb3A0&t=18s) for the video on YT) and the related fastbook, the Jupyter-Book from fast.ai. The lecture is based on the second chapter of the book. 

The lesson presents key concepts, guiding the transformation of our model prototype from Lesson 1 into an end-to-end deep learning system for deployment in a production environment.

Lesson structure:

In [ ]:
# Uncomment to install the necessary packages  
# !pip install -r requirements.txt